In [1]:
#embedding 테스트 - 완료
##one-hot으로 할 필요가 없음.
##embedding_size도 마음대로 결정할 수 있음.
##random init으로 해도됨.
##input의 embedding을 training할수도 있고 안할수도 있음
##Y는 embedding 할 필요 없음. 그냥 target class shape만 맞춰서 loss를 구하면 됨

#file에서 읽어와서 input으로 넣어보기 - 완료
##이미 index화 되어 있는 파일 읽어오기 - 완료
##dictionary file로부터 dic 생성 - 완료
##embedding 활용 - 완료

#saver 적용 - 완료

#tensorboard 적용

#padding 적용

#batch 적용

#bidirectional rnn 적용

from __future__ import print_function

import tensorflow as tf
import numpy as np
import pprint as pp
import os
import random

from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility

In [2]:
checkpoint_path = 'data/dev/ckpt_birnn/'

def read_sentences_from_file(file_path):
    sentence_file = open(file_path, 'r')
    sentences = []
    while True:
        line = sentence_file.readline()
        if not line:
            break;
        sentences.append(line.replace('\n',''))
    sentence_file.close()
    return sentences

def read_sentence_indexes_from_file(file_path):
    sentence_index_file = open(file_path, 'r')
    sentence_indexes = []
    print("start reading ",file_path)
    line_count = 0
    while True:
        line = sentence_index_file.readline()
        if not line:
            break;
        sentence_indexes.append([int(c) for c in line.replace('\n','').split()])
        if line_count % 100000 == 0:
            print("reading line ",line_count)
        line_count+=1
        
    sentence_index_file.close()
    return sentence_indexes

def char_tokenizer(sentence): #sentence는 byte로 들어옴
    sentence = sentence.replace('<english>','a').replace('<number>','0')
    return [char for char in sentence] #그 후 최종 다시 byte로 변환한다

def create_dic(sentences, save_file_path):
    dic = []
    file = open(save_file_path, 'w+')
    for sentence in sentences:
        tokens = char_tokenizer(sentence)
        for token in tokens:
            if not (token in dic):
                dic.append(token)
                file.write(token+'\n')
    file.close
    return dic
    
def load_dic(dic_file_path):
    dic_file = open(dic_file_path, 'r')
    rev_vocab_dic = []
    while True:
        line = dic_file.readline()
        if not line:
            break;
        rev_vocab_dic.append(line.replace('\n',''))
    vocab_dic = {w: i for i, w in enumerate(rev_vocab_dic)}
    return vocab_dic, rev_vocab_dic

def sentences_to_indexes(sentences, vocab_dic):
    indexes = []
    for sentence in sentences:
        tokens = char_tokenizer(sentence)
        indexes.append([vocab_dic[char] for char in tokens])
    return indexes

In [3]:
##직접 dic을 구축하고, index를 만드려면 아래의 코드를 사용
# chosung_sentences = read_sentences_from_file('data/dev/namuwiki_chosung_dev.txt')
# target_sentences = read_sentences_from_file('data/dev/namuwiki_sentences_dev.txt')

# _ = create_dic(chosung_sentences, 'data/dev/chosung_dic_dev')
# _ = create_dic(target_sentences, 'data/dev/target_dic_dev')

# chosung_vocab_dic, chosung_rev_vocab_dic = load_dic('data/dev/chosung_dic_dev')
# target_vocab_dic, target_rev_vocab_dic = load_dic('data/dev/target_dic_dev')

# chosung_sentence_indexes = sentences_to_indexes(chosung_sentences, chosung_vocab_dic)
# target_sentence_indexes = sentences_to_indexes(target_sentences, target_vocab_dic)

In [4]:
##이미 만들어진 dic과 index를 사용하려면 아래의 코드를 사용
chosung_sentence_indexes =  read_sentence_indexes_from_file('data/train/namuwiki_chosung.txt.ids10000')
target_sentence_indexes =  read_sentence_indexes_from_file('data/train/namuwiki_sentences.txt.ids10000')
chosung_vocab_dic, chosung_rev_vocab_dic = load_dic('data/dev/vocab_dev.from')
target_vocab_dic, target_rev_vocab_dic = load_dic('data/dev/vocab_dev.to')

start reading  data/train/namuwiki_chosung.txt.ids10000
reading line  0
reading line  100000
reading line  200000
reading line  300000
reading line  400000
reading line  500000
reading line  600000
reading line  700000
reading line  800000
reading line  900000
start reading  data/train/namuwiki_sentences.txt.ids10000
reading line  0
reading line  100000
reading line  200000
reading line  300000
reading line  400000
reading line  500000
reading line  600000
reading line  700000
reading line  800000
reading line  900000


In [5]:
hidden_size = 30
num_classes = len(target_rev_vocab_dic)
embedding_size = 30
sequence_length = 50
learning_rate = 0.005
batch_size = 256

In [6]:
def get_batch():
    dataX = []
    dataY = []
    output_weights = []
    sequence_length_arr = []

    for _ in range(batch_size):
        i = random.randrange(0,len(chosung_sentence_indexes))
        output_weight = np.ones(sequence_length, dtype=np.float32)
        chosung_indexes = list(chosung_sentence_indexes[i])
        target_indexes = list(target_sentence_indexes[i])
        if len(chosung_indexes) > sequence_length:
            chosung_indexes = chosung_indexes[:sequence_length]
            target_indexes = target_indexes[:sequence_length]
            sequence_length_arr.append(sequence_length)
        else:
            sequence_length_arr.append(len(chosung_indexes))
            pad_size = sequence_length - len(chosung_indexes)
            chosung_indexes += [0]*pad_size #padding
            target_indexes += [0]*pad_size #padding
            for j in range(pad_size):
                output_weight[sequence_length-1-j] = 0.0 #padding인 경우 weight를 0으로 초기화
        dataX.append(chosung_indexes) 
        dataY.append(target_indexes)
        output_weights.append(output_weight)
        
    return dataX, dataY, output_weights, sequence_length_arr


In [7]:
X = tf.placeholder(tf.int32, [None, sequence_length], name="chosung_sentences")
Y = tf.placeholder(tf.int32, [None, sequence_length], name="target_sentences")
weights = tf.placeholder(tf.float32, [None, sequence_length], name="seqence_weights")
output_sequence_size = tf.placeholder(tf.int32, [None], name="output_sequence_size")

# learning_rate_var = tf.Variable(float(learning_rate), trainable=False, dtype=tf.float32)
# learning_rate_decay_op = learning_rate_var.assign(learning_rate_var * 0.99)

In [8]:
print(num_classes)
embeddings = tf.Variable(
    tf.random_uniform([len(chosung_vocab_dic), embedding_size], -1.0, 1.0), trainable=True, name="chosung_embeddings")
embedded_input = tf.nn.embedding_lookup(embeddings, X, name="embedded_chosung_sentences")
print(embedded_input)

3548
Tensor("embedded_chosung_sentences:0", shape=(?, 50, 30), dtype=float32)


In [9]:
# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 8, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, embedded_input, dtype=tf.float32, sequence_length=output_sequence_size)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size], name="reshaped_rnn_output")
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes], name="predictions")

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights, name="sequence_loss")
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [10]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state(checkpoint_path)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("Created model with fresh parameters.")
    sess.run(tf.global_variables_initializer())

Reading model parameters from data/dev/ckpt_birnn/chosung_rnn.ckpt


In [ ]:
loss_history = []
step_loss = 0.0
for i in range(1000000):
    dataX, dataY, output_weights, sequence_length_arr = get_batch()
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY, weights: output_weights, output_sequence_size: sequence_length_arr})
    step_loss += (l / 100)
    if (i+1) % 100 == 0:
        print('step:',i,' loss:',step_loss)
        loss_history.append(step_loss)
        step_loss = 0.0
        
#         if len(loss_history) > 2 and step_loss > max(loss_history[-3:]):
#             sess.run(learning_rate_decay_op)
        
        dataX, dataY, output_weights, sequence_length_arr = get_batch()
        results = sess.run(outputs, feed_dict={X: dataX, output_sequence_size: sequence_length_arr})
        for test_index in range(10):
            index = np.argmax(results[test_index], axis=1)
            print(''.join([target_rev_vocab_dic[t] for t in index]))
        print('----------')
        checkpoint_file_path = os.path.join(checkpoint_path, "chosung_rnn.ckpt")
        saver.save(sess, checkpoint_file_path)
        print('saved checkpoint')


step: 99  loss: 1.78616183758
이본 최대의 최피스 전 한나의 사용에서의서 보혀하는 말한치도이 카트고를이 사용에서지판 카메
0년 0월 0일부터 레프렉 비트 경리부 이프이 카버버키를 사리되었다 마마마마마마마마마마마마
이본 대회에서는 사작 이터뷰로 a를 보이하는 팀자이 있는데 0월로 통고하 정면차에 하다 이
전목 그리이로이 하지 않지에 이는 이전 이전에 가우다 프함 마마마마마마마마마마마마마마마마마
a a a 나이도 전체가 a a 나이도로 카리어하여 이리이 대는 모이지서 이 곡정를 대시하
다 a 나자한에로는 정은 가이하면 이이로 a0에지 a의 카게 최음는지는 이는다 마마마마마마
사간 최가에는 이이이 모이이지 프이하지 못하지만 0대 대어 0년 0월 0일 대일 가장에로 
 타수 비치하는의 모찬 이상한서수 그화사수 전반하서수 한국적a0회국전a 라즈가타프부에 레호
그본적으로 이드 모이는 특마널에서 전원 때는 다장인 사태하 정투로 하는 보송 마마마마마마마
이런스가 전신의 전신 보행를면 그은 가리 편이다 이이 모리면 한면 대어을 통이들 마마마마마
----------
saved checkpoint
step: 199  loss: 1.78945420861
하지는 그장 크게 이리다어 기장 크 보해가는 a0의다 사리를는 a0보다 0위 대 것대 있고
이기에 한률이 최보하다면 사수 이이 전 리벨에 자은트가 전기에 모 그대로 이어서 가리 대어
이에서 이기하던 이 대 보전에 있리서다로에에 이터에크의의 포로 대미트를이서 사리이크의의으 
물지한이 이니 것체들면에 사작하여서는 이전히 때같다은 있다로이 자행되다는 것이 많습하다라 
 a a a a a 0a0 최반부에는 0 지도의 포이하 프턴이 나오다가 한보에 가이서를이 
물론 그 기은는 이상트리 대어서 이상적 사황 이이로 통수 수황다면로 사수 전행이 가리부리고
0 0a0 김우 미이는한 특히이 쓰는 차가로 이지로면 특이버지에 모이 수각하는 중에트에 모
그리한 이이 전체 모스을 이해 모이이 이끌 수명하다이 보 이리이로는 부스자 사황

In [ ]:
print(len(results)) #batch size
print(len(results[0])) #sequence length
print(len(results[0][0])) #vocab_size

print(batch_size)
print(sequence_length)
print(len(target_vocab_dic))


In [ ]:
#embedding 연습!!
import tensorflow as tf

embeddings = tf.Variable(
    tf.random_uniform([num_classes, 50], -1.0, 1.0))

train_inputs = tf.placeholder(tf.int32, shape=[3,2])
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
ems, inputs,embd = sess.run([embeddings,train_inputs,embed], feed_dict={train_inputs:[[1,2],[2,3],[3,4]]})

for i in ems:
    print(i)
print('-----------')
print(inputs)
print('-----------')
print(embd)